# Traffic Video Analytics with YOLO

## Library Imports

In [25]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

## Zone and Parameter Setup

In [26]:
def setup_zones():
    polygon = np.array([
        [4, 350], [176, 285], [202, 258], [561, 201], [650, 188], [753, 140],
        [948, 167], [948, 225], [1006, 299], [1084, 363], [1182, 452], [1275, 461],
        [1278, 653], [1278, 711], [1166, 716], [974, 716], [730, 717], [561, 717],
        [554, 638], [722, 590], [807, 511], [819, 450], [777, 377], [683, 335],
        [586, 322], [476, 327], [345, 365], [278, 421], [242, 459], [223, 488],
        [228, 536], [284, 595], [368, 627], [536, 639], [547, 641], [558, 714],
        [10, 716]
    ], dtype=np.int32)

    classes = [2, 5, 7]

    line_zones = [
        sv.LineZone(sv.Point(649, 187), sv.Point(926, 229), triggering_anchors=(sv.Position.CENTER,)),
        sv.LineZone(sv.Point(1214, 518), sv.Point(1114, 714), triggering_anchors=(sv.Position.CENTER,)),
        sv.LineZone(sv.Point(1, 385), sv.Point(181, 271), triggering_anchors=(sv.Position.CENTER,))
    ]

    polygon_zone = sv.PolygonZone(polygon=polygon, triggering_anchors=(sv.Position.CENTER,))
    return polygon, classes, line_zones, polygon_zone

## Model and Annotator Initialization

In [27]:
def initialize_tools():
    model = YOLO("yolo11m.pt")
    tracker = sv.ByteTrack(minimum_consecutive_frames=3)
    tracker.reset()
    smoother = sv.DetectionsSmoother()

    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator(text_color=sv.Color.BLACK)
    trace_annotator = sv.TraceAnnotator(trace_length=60)

    line_zone_annotator = sv.LineZoneAnnotator(
        text_scale=0.8,
        text_orient_to_line=True,
        custom_in_text="in ",
        custom_out_text="out ",
        display_out_count=False,
        display_in_count=False
    )

    line_zone_annotator_multiclass = sv.LineZoneAnnotatorMulticlass(
        text_scale=0.8,
        text_thickness=2,
        table_margin=20
    )

    return model, tracker, smoother, box_annotator, label_annotator, trace_annotator, line_zone_annotator, line_zone_annotator_multiclass

## Frame Processing Function

In [28]:
def process_frame(frame, model, tracker, smoother, polygon, classes, polygon_zone, line_zones,
                  box_annotator, label_annotator, trace_annotator, line_zone_annotator, line_zone_annotator_multiclass):
    
    resized_frame = cv2.resize(frame, (1280, 720))
    result = model(resized_frame, device="cuda", verbose=False, imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(result)
    detections = detections[polygon_zone.trigger(detections)]
    detections = detections[np.isin(detections.class_id, classes)]
    detections = tracker.update_with_detections(detections)
    detections = smoother.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {model.names[class_id]} {confidence:0.2f}"
        for tracker_id, class_id, confidence in zip(
            detections.tracker_id, detections.class_id, detections.confidence
        )
    ]

    for zone in line_zones:
        zone.trigger(detections=detections)

    annotated_frame = resized_frame.copy()
    annotated_frame = sv.draw_polygon(annotated_frame, polygon, sv.Color.RED, thickness=2)
    annotated_frame = box_annotator.annotate(annotated_frame, detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections, labels)
    annotated_frame = trace_annotator.annotate(annotated_frame, detections)

    for zone in line_zones:
        annotated_frame = line_zone_annotator.annotate(annotated_frame, zone)

    annotated_frame = line_zone_annotator_multiclass.annotate(
        annotated_frame,
        line_zones=line_zones,
        line_zone_labels=["Center Line", "Right Line", "Left Line"]
    )

    return annotated_frame

## Video Processing Function

In [29]:
def process_video(video_file_path, take_frame=False):
    polygon, classes, line_zones, polygon_zone = setup_zones()
    model, tracker, smoother, box_annotator, label_annotator, trace_annotator, line_zone_annotator, line_zone_annotator_multiclass = initialize_tools()

    frame_generator = sv.get_video_frames_generator(source_path=video_file_path, stride=1)
    for frame in frame_generator:
        annotated_frame = process_frame(
            frame, model, tracker, smoother, polygon, classes, polygon_zone, line_zones,
            box_annotator, label_annotator, trace_annotator, line_zone_annotator, line_zone_annotator_multiclass
        )

        if take_frame:
            cv2.imwrite("frame.jpg", annotated_frame)
            break

        cv2.imshow('Processed Video', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()

## Run the Processing

In [31]:
video_path = "traffic-india.mp4"  # Replace with your actual path
process_video(video_path, take_frame=False)